<a href="https://colab.research.google.com/github/markaaronslater/NMT/blob/master/NMT_driver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!pip install subword-nmt # for segmenting words into subwords
!pip install stanza # for tokenizing corpus and tagging with morphological data
!pip install sacremoses # for detokenizing model predictions
!pip install sacrebleu # for evaluation

In [ ]:
# make sure using GPU
# (Runtime -> Change runtime type -> Hardware accelerator = GPU).
!nvidia-smi

In [ ]:
# recommended: place cloned NMT folder in Google drive folder 'My Drive':
path = '/content/gdrive/My Drive/NMT/'
#corpus_path = path + 'corpuses/iwslt16_en_de/'
corpus_path = path + 'data/iwslt/en-de/'
config_path = path + 'configs/'

### REQUIRED: ###
# create a folder inside of checkpoints, named <model_name>. this will hold all checkpoints for the model, its per-epoch training stats,
# and files holding its greedy dev set predictions after each epoch.
model_name = 'big_relu/' # your model name here. remember to ensure desired settings are set in config files of NMT/configs/ !!!
checkpoint_path = path + 'checkpoints/' + model_name

In [ ]:
%cd /content/gdrive/My Drive/NMT

In [ ]:
from src.preprocessing.apply_stanza_processors import apply_stanza_processors
from src.preprocessing.truecase import truecase_corpuses
from src.preprocessing.preprocess import construct_model_data
from src.preprocessing.corpus_utils import read_corpus, get_references
from src.import_configs import import_configs
from src.train import train, load_checkpoint
from src.predict import predict
from src.evaluate import evaluate
from src.model_utils import load_pretrained
from src.translate import translate
import stanza
from subword_nmt.apply_bpe import BPE

In [ ]:
# step 1 - tokenize corpuses, and tag with morphological data.
apply_stanza_processors("train.de", "train.en", "dev.de", path=corpus_path)

In [ ]:
# step 2 - true-case corpuses using linguistic heuristics that leverage morphological
# data produced by morphological data tagger.
truecase_corpuses("train.de", "train.en", "dev.de", corpus_path=corpus_path)

In [ ]:
# import vocab, training, and model hyperparameter settings from configuration files.
hyperparams = import_configs(config_path=config_path)
print(hyperparams)

In [ ]:
# step 3 - segment words of corpuses into subwords (skip this cell if using a word-level vocabulary).
num_merge_ops = hyperparams["num_merge_ops"]
vocab_threshold = hyperparams["vocab_threshold"]
truecased_path = corpus_path + 'truecased/'
segmented_path = corpus_path + 'subword_segmented/'

!bash ./src/preprocessing/subword_joint.sh $num_merge_ops $vocab_threshold "$truecased_path" "$segmented_path"

In [ ]:
# step 4 - build intelligently batched sets of tensors that can be directly passed to model.
construct_model_data("train.de", "train.en", "dev.de", hyperparams=hyperparams, corpus_path=corpus_path, checkpoint_path=checkpoint_path)

In [ ]:
# step 5 - instantiate and train model.
model, loss = train(checkpoint_path=checkpoint_path)

In [ ]:
# prepare pretrained model for end-to-end inference.
stanza.download(lang='de', processors='tokenize,mwt,pos')
stanza_de_processor = stanza.Pipeline(lang='de', processors='tokenize,mwt,pos', tokenize_no_ssplit=True, tokenize_batch_size=64, mwt_batch_size=200, pos_batch_size=10000)

In [ ]:
# step 6 - evaluate test set predictions.
translator, model_data = load_pretrained(checkpoint_path)
src_word_to_idx = model_data["src_word_to_idx"]
idx_to_trg_word = model_data["idx_to_trg_word"]

# optional: first observe beam search predictions of best model on dev set:
# (during training, used greedy search).
# bleu should improve by ~1.5
dev_batches = model_data["dev_batches"]
dev_references = model_data["references"]
dev_translations, preds_time, post_time = predict(translator, dev_batches, idx_to_trg_word, checkpoint_path)
bleu = evaluate(dev_translations, dev_references)
print(round(bleu, 2))
print(preds_time)




# evaluate model on test set.
bpe = BPE(open(corpus_path + 'subword_segmented/bpe_codes', 'r'), vocab=set(src_word_to_idx))
test_path = path + 'data/iwslt/en-de/'
test_set = read_corpus('test.de', path=test_path)
test_references = get_references(path=test_path, dev=False)
translations = translate(test_set, stanza_de_processor, translator, src_word_to_idx, idx_to_trg_word, bpe)
print(evaluate(translations, test_references))

In [ ]:
# b) run unit tests to show correctness of model implementations
# can run each separately, or discover and run all at once (see below)

In [ ]:
# allow ~5 min to run all model variant tests, each of which trains for 100 epochs.
!python -m pytest

In [ ]:
!python -m pytest unittests/test_batches.py # ensure intelligent batching procedure is correct

In [ ]:
!python -m pytest -s -v unittests/test_model.py::test_default_word_model

In [ ]:
!python -m pytest -s -v unittests/test_model.py::test_tie_weights

In [ ]:
!python -m pytest -s -v unittests/test_model.py::test_subword_tie_weights

In [ ]:
!python -m pytest -s -v unittests/test_model.py::test_outdrop

In [ ]:
!python -m pytest -s -v unittests/test_model.py::test_default_subword_model

In [ ]:
# before running this cell, ensure using cpu
# (Runtime -> Change runtime type -> Hardware accelerator = None).
# allow several minutes for this test to run.
#!python -m pytest -s -v unittests/test_model.py::test_default_word_model_cpu

In [ ]:
!python -m pytest -s -v unittests/test_model.py::test_uni_no_attn

In [ ]:
!python -m pytest -s -v unittests/test_model.py::test_layer_to_layer_uni_no_attn

In [ ]:
!python -m pytest -s -v unittests/test_model.py::test_final_to_first_uni_no_attn

In [ ]:
!python -m pytest -s -v unittests/test_model.py::test_dropout

In [ ]:
!python -m pytest -s -v unittests/test_model.py::test_no_tying

In [ ]:
!python -m pytest -s -v unittests/test_model.py::test_no_attn_no_tying